## Link http://tiny.cc/pxo6yz

In [ ]:
import numpy as np
from scipy.signal import correlate2d
from scipy import ndimage

## Implement the following functions step-by-step

In [ ]:
def correlation_func(input_f: np.array, kernel_h: np.array, padding: bool=False, padding_mode: str=None) -> np.array:
  """ +0.1
  input_f: 2D array, input array
  kernel_h: 2D array, kernel array
  padding: boolean, True/False, padding or not
  padding_mode: zero/replicator/reflection

  Output: output_g, 2D array
  """
  # Get shape of input
  N, M = input_f.shape
  # Get shape of kernel
  kn, km = kernel_h.shape
  # Calculate r1, r2 (2*r1+1=kn, 2*r2+1=km)
  r1, r2 = kn//2, km//2

  if padding:
    if padding_mode == 'zero':
      mode='constant'
    elif padding_mode == 'replicator':
      mode='edge'
    elif padding_mode == 'reflection':
      mode='symmetric'
    else:
      raise ValueError(f'Unknown padding mode {padding_mode}')

    # Do padding,
    # https://numpy.org/doc/stable/reference/generated/numpy.pad.html
    n_pad, m_pad = r1, r2
    input_f_padded = np.pad(input_f, ((n_pad, n_pad), (m_pad, m_pad)), mode=mode)
    # Shape of output_g
    gN, gM = N, M
  else:
    # No padding => shape of output_g will be smaller than the input
    gN, gM = N-kn+1, M-km+1
    n_pad, m_pad= 0, 0
    input_f_padded = input_f

  # Initialize the output array with zeros
  output_g = np.zeros((gN, gM), dtype=np.float64)

  # Do Calculation, for loop
  for u in range(gN):
    for v in range(gM):
      # (u, v) location in the output array
      # (u_st, v_st) corresponding location in padded input array
      u_st, v_st = u + r1, v + r2
      # Get local data on input array
      current_local = input_f_padded[u_st - r1: u_st + r1+1, v_st - r2: v_st +r2+1]
      # Multiply, sum, assign to output_g
      output_g[u, v] = np.sum(np.multiply(current_local, kernel_h))

  # Return the output
  return output_g

def create_gaussian_kernel(kernel_h: int, kernel_w: int, s: float=0.01) -> np.array:
  """
  Create Gaussian kernel for filtering (+0.05)

  kernel_h: height of kernel
  kernel_w: width of kernel
  s: standard deviation of Gaussian distribution

  Output: gauss_kernel of size (kernel_h, kernel_w)
  """
  # Calculate r1, r2
  r1, r2 = kernel_h//2, kernel_w//2

  # Initialize the output with zero values
  gauss_kernel = np.zeros((kernel_h, kernel_w), dtype=np.float64)

  # Do calculation, for loop
  for x in range(-r1, r1+1):
    for y in range(-r2, r2+1):
      gauss_kernel[x+r1, y+r2] = (1/(2*np.pi*(s**2))) * np.exp(-(x**2 + y**2)/(2*(s**2)))

  # Do normalization so the sum of all values equal to 1, to avoid values in kernel too large/small
  gauss_kernel = gauss_kernel/np.sum(gauss_kernel)

  return gauss_kernel

def create_mean_kernel(kernel_h: int, kernel_w: int) -> np.array:
  """
  Create Mean kernel for filtering (+0.05)
  kernel_h: height of kernel
  kernel_w: width of kernel

  Output: mean_kernel of size (kernel_h, kernel_w)
  """
  # Do something
  mean_kernel = (1/(kernel_h*kernel_w)) * np.ones((kernel_h, kernel_w), dtype=np.float64)
  return mean_kernel

def median_filtering(input_f: np.array, kernel_h: int, kernel_w: int, padding: bool=False, padding_mode: str=None) -> np.array:
  """
  Do Median filtering (+0.1)

  input_f: 2D array, input array
  kernel_h, kernel_w: int values, height and width of kernel
  padding: boolean, True/False, padding or not
  padding_mode: zero/replicator/reflection

  Output: output_g, 2D array
  """

  # Get shape of input
  N, M = input_f.shape
  # Get shape of kernel
  kn, km = kernel_h, kernel_w
  # Calculate r1, r2 (2*r1+1=kn, 2*r2+1=km)
  r1, r2 = kn//2, km//2

  if padding:
    if padding_mode == 'zero':
      mode='constant'
    elif padding_mode == 'replicator':
      mode='edge'
    elif padding_mode == 'reflection':
      mode='symmetric'
    else:
      raise ValueError(f'Unknown padding mode {padding_mode}')

    # Do padding,
    # https://numpy.org/doc/stable/reference/generated/numpy.pad.html
    n_pad, m_pad = r1, r2
    input_f_padded = np.pad(input_f, ((n_pad, n_pad), (m_pad, m_pad)), mode=mode)
    # Shape of output_g
    gN, gM = N, M
  else:
    # No padding => shape of output_g will be smaller than the input
    gN, gM = N-kn+1, M-km+1
    n_pad, m_pad= 0, 0
    input_f_padded = input_f

  # Initialize the output array with zeros
  output_g = np.zeros((gN, gM), dtype=np.float64)

  # Do Calculation, for loop
  for u in range(gN):
    for v in range(gM):
      # (u, v) location in the output array
      # (u_st, v_st) corresponding location in padded input array
      u_st, v_st = u + r1, v + r2
      # Get local data on input array
      current_local = input_f_padded[u_st - r1: u_st + r1+1, v_st - r2: v_st +r2+1]
      # Get median and assign
      output_g[u, v] = np.median(current_local)

  # Return the output
  return output_g

In [ ]:
ksize_w, ksize_h = 3, 3
gauss_s = 0.5

mean_kernel = create_mean_kernel(ksize_h, ksize_w)
gauss_kernel = create_gaussian_kernel(ksize_h, ksize_w, gauss_s)

print('Mean kernel\n', mean_kernel)
print('Gauss kernel\n', gauss_kernel)

Mean kernel
 [[0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]]
Gauss kernel
 [[0.01134374 0.08381951 0.01134374]
 [0.08381951 0.61934703 0.08381951]
 [0.01134374 0.08381951 0.01134374]]


In [ ]:
input_arr = np.random.randn(32, 48)
kernel_arr = np.random.randn(3,5)

corr_zp = correlation_func(input_arr, kernel_arr, padding=False)
scipy_zp = correlate2d(input_arr, kernel_arr, mode='valid')
print('Checking zero padding corr:', np.sum(np.abs(corr_zp-scipy_zp)) < 1e-8)

corr_refp = correlation_func(input_arr, kernel_arr, padding=True, padding_mode='reflection')
scipy_refp = correlate2d(input_arr, kernel_arr, mode='same', boundary='symm')
print('Checking zero padding corr: ', np.sum(np.abs(corr_refp-scipy_refp)) < 1e-8)

Checking zero padding corr: True
Checking zero padding corr:  True


In [ ]:
kernel_h, kernel_w = 5, 7
median_repl = median_filtering(input_arr, kernel_h, kernel_w, padding=True, padding_mode='replicator')
median_scirepl = ndimage.median_filter(input_arr, size=(kernel_h, kernel_w), mode='nearest')

print('Checking replicator padding median:', np.sum(np.abs(median_repl-median_scirepl)) < 1e-8)

median_refl = median_filtering(input_arr, kernel_h, kernel_w, padding=True, padding_mode='reflection')
median_scirefl = ndimage.median_filter(input_arr, size=(kernel_h, kernel_w), mode='reflect')

print('Checking reflection padding median:', np.sum(np.abs(median_refl-median_scirefl)) < 1e-8)

median_zp = median_filtering(input_arr, kernel_h, kernel_w, padding=True, padding_mode='zero')
median_scizp = ndimage.median_filter(input_arr, size=(kernel_h, kernel_w), mode='constant')

print('Checking zero padding median:', np.sum(np.abs(median_zp-median_scizp)) < 1e-8)

Checking replicator padding median: True
Checking reflection padding median: True
Checking zero padding median: True
